# AdaptER-Covid19

## Introduction

Accurate epidemiological simulations are a key tool for understanding, responding to and managing the Covid-19 pandemic. While understanding health-outcomes of policy interventions is rightly the primary focus of such simulations, there are clear benefits to a holistic simulation that takes into account economic dynamics as well. Created for this purpose, AdaptER-Covid19 (Adaptive Economic Response) is an economic simulation that integrates with the OpenABM-Covid19 simulator in order to jointly model health and economic outcomes of epidemics, with a particular focus on Covid-19.

AdaptER-Covid19 takes as input from the epidemiological part of the model timeseries describing the simulated course of the epidemic (e.g. number of symptomatic individuals) as well as exogenous variables (e.g. whether a lockdown is in place in the given scenario) and simulates the impact on key economic metrics such as GDP, corporate bankruptcy rate and individual insolvency rate. The main focus of the first version of the model is on the short-term effects of labour supply constraints on economic output, as well as second-order effects such as corporate bankruptcies and the longer-term effects these have on capital stock and employment. For this purpose, the UK economy is broken down by sector and region to model local and sector-specific effects of both the epidemic and the policy response. Demand-side effects, such as impact on consumption and investment are not modelled in detail; this will be addressed in future work.

It should also be noted that AdaptER-Covid19 is a simulator rather than a forecast. Its outputs are dependent on both the mathematical model as well as a host of parameter choices on which limited data is available (e.g. worker productivity by region and sector when limited to working from home) and which may vary considerably across scenarios. As such, simulation results from AdaptER-Covid19 may be used to draw qualitative conclusions when comparing different scenarios rather than for quantitative estimates or inference of absolute numbers. To allow the user to properly evaluate simulation results, we describe the current model in detail below.

## Model Overview

To limit complexity, the overall economic simulator is broken down into four linked components. The first three model the impact of the epidemic:

* an *input-output model* of the economy that links primary and intermediate inputs to production, on the one hand, to intermediate and final uses of production, on the other hand, on a sector by sector basis and is used in particular to model the effect of constraining labour supply on GDP,
* a *corporate bankruptcy model* which estimates corporate default rates over time, based on data from the input-output model as well as assumptions around corporate cash flow,
* an *individual solvency model* which estimates risk of individual insolvencies, by region and sector, based on both corporate bankruptcies as well as overall GDP.

Taken together, the above three models constitute a simulation of the short-term shock to the economy caused by the epidemic. Moreover, the destruction of capital through an excess rate of corporate defaults is used to discount GDP going forward. However, this approach is inherently limited to modelling a shortfall of GDP with respect to a baseline and does not account for growth. For this purpose, we add

* a *long-term trend model* which models growth dynamics of the economy based on fixed long-term growth assumptions

and which is overlaid on the combined output of the first three models.

Before we go into further detail on these four component models, we point out key **limitations** in the current approach:

* Some quantities that are already modelled in some of the component models are currently not fully linked with other components. For example, financial distress of households from the individual solvency model should be used to constrain household consumption in the final use section of the input-output model. Similarly, capital destruction given by the corporate bankruptcy model should directly affect capital stock in the primary input section of the input-output model. These feedback loops are currently modelled using ad-hoc GDP discounting. A deeper integration along the lines described above is future work.

* Consumption patterns are not represented in the model yet. Thus, there are a wide range of effects of economic import that are not currently possible to simulate, for example: Consumer reluctance to go to restaurants as case numbers increase, even if a lockdown is not in place. Pent-up demand leading to an increase in spending after a prolonged lockdown is lifted. Shift in demand towards delivery services and home entertainment. Increase in the savings rate across households in response to economic downturn.

* Similarly, changes in investment behaviour of corporates are not simulated with granularity, apart from overall destruction of capital stock due to defaults. Effects that cannot be simulated in detail include: Investment in technology to facilitate working from home. Investment in delivery services and online ordering and divestment from brick-and-mortar stores. Deleveraging of corporates that do not default.

* Government spending is not currently modelled as a separate source of demand, but rather assumed to be constant. In particular, specific economic stimulus packages are not taken into account and will be addressed in future work.

* As mentioned previously, the model currently combines a model of the short-to-medium term downside impact of the epidemic with a simple long-term growth trend model. Modelling the dynamics of demand as described above properly will give a more detailed simulation of the return to growth in the long term.

* The UK government has deployed a wide range of economic policies. Some of these, such as furloughing of workers and tax deferral are included to a first approximation. However, a wide range of other measures, from mortgage holidays and Statutory Sick Pay to the Covid-19 Corporate Financing Facility and the Coronavirus Business Interruption Loan Scheme, are not currently modelled.

* Labour and capital markets are not currently modelled. For example, the dynamics of employees switching firms or industries and of investors redeploying capital is not modelled. Prices for goods and services are assumed to be constant.

Addressing these limitations is left for future work. However, it is essential to keep current limitations of the simulator in mind when interpreting results. Also see the section on parameters below.

## Model Components

### Input-Output Model

The foundation of our model of sectoral interdependencies is the European System of Accounts and in particular the supply and use tables and input-output analytical tables (IOATs) as set out by Eurostat [1] and applied to the UK by the ONS [2]. This framework allows to express total demand for a given product, on the one hand, as the sum of primary inputs and intermediate inputs to production at basic prices and, on the other hand, as the sum of intermediate demand and final uses. Primary inputs are, broadly, broken down into imports, taxes, compensation and operating surplus. Final uses are broken down into final consumption, capital formation and exports. This accounting perspective provides a common reference points that can be referred to across component models, from both a production and consumption perspective. In particular, the matrix of intermediate demand reflects sectoral interdependencies in the UK economy. Note that while IOATs typically categorize demand by product, we assume a categorization by sector, according to the UK's Standard Industrial Classification (SIC 2007), particularly A21 in NACE Rev. 2, see [2, Annex A].

To turn this accounting framework into a model suitable for simulation, we need to specify a production function that, in particular, allows for substitution of one input to production with another, at a cost. Following [3], we adopt a Cobb-Douglas production function that takes intermediate inputs as well as imports, labour and (fixed) capital as arguments. The IOATs are used to calibrate weights in the Cobb-Douglas function. Here we make the crucial assumption that prices for goods are constant. The price of labour responds to exogenous changes to both quantity of labour and productivity imposed by illness or government measures such as lockdown, by region, sector and age-group. This is in contrast to the modelling of capital, where we assume that the stock of capital is fixed at a level given exogenously and the return on capital varies according to the model. Final use is capped at a level given exogenously. See also [4] for further background on modelling of network effects in the economy.

The input-output model currently does not have an explicit representation of time. It takes as parameters the supply of labour, as derived from the output of the epidemiological model, and simulates the economic output as a fraction of pre-crisis GDP. (As mentioned above, other exogenous parameters such as demand and stock of capital can be used in future work.) The model is run repeatedly as parameter values change over time and the output is then integrated to estimate GDP over time.

Note that the code base currently contains a simplified version of the above model based on linear programming (LP) which scales economic output linearly for a given supply of labour by sector, region and age. A full nonlinear programming (NLP) implementation including the Cobb-Douglas production function is in development.

### Corporate Bankruptcy Model

The objective of the bankruptcy model is to model the proportion of companies going insolvent and corresponding reduction to GDP  over time. In this context, it is crucial to distinguish between the probability of a specific company defaulting and the proportion of companies in the population that default. For this purpose we apply the log-logistic (Fisk) distribution to model the variation in number of days until companies go insolvent; this distribution is defined by two parameters (shape and scale) and is commonly used in survival analysis, including in the context of business survival [5,6]. 

The shape parameter of the distribution is assumed to be constant across sectors and consistent before, during, and after the epidemic. The scale parameter is modelled as a function of the mean operating surplus as well as the mean cash buffer that firms have in a given sector. Operating surplus is taken as given by the input-output model, with additional work underway to leverage the granular modelling of primary inputs available in the input-output model. The cash buffer is treated as exogenous parameter that needs to be calibrated. Care is taken to distinguish between small and medium enterprises (SMEs) and large corporations as the survival rates and GDP contributions are quite different for these groups of companies.

One important limitation is that the model currently does not account for interest payments, which form a crucial fixed payment, especially for highly-leverages firms and will be addressed in future work. Another limitation, already mentioned above, is that government support programs are currently not modelled explicitly.

### Individual Insolvency Model

We model the proportion of individual insolvency over time, as a function of credit quality by region.

$$P_r(t)=P(\text{Credit Score}_r(t) < \text{Default Threshold})$$

We assume Credit Score per region at any given time follows a mixture of Gaussian distribution, where 
- Each Guassian component represents one employed sector, and one decile of the population in that sector 
- The weight of each component equals the number of workers per region per sector, divided by the number of deciles
- The shape parameter of the all components is fixed fixed per region
- The location parameter is a piecewise linear function of the initial credit quality before the onset of the simulation, the cumulative balance of savings the individual has, and the current change in balance.

$$\text{Credit Score}_r(t) \sim 
\sum_l \sum_s w_{r, s}^l \mathcal{N}(m_{r, s}^l(t), \sigma_r)$$

$$w_{r^{*}, s^{*}}^l = 
\frac{1}{L} * \frac{N_{r^{*}, s^{*}}}{\sum_{r, s}N_{r, s}} 
\thinspace \forall l \thinspace \text{in  \{1,2,...,L\}}$$

$$m_{r, s}^l(t) = 
\text{Credit Score}_r(0) + 
\alpha * \delta \text{Balance}_{r,s} ^ l(t) + 
\beta * \min(\text{Balance}_{r, s}^l(t), 0)$$

The average balance per region, sector, decile itself takes savings at the start of the simulation as well as cumulative earnings over the course of the simulation into account. We analyze earnings and spendings separately to study the diffrent impact received. 

$$\text{Balance}_{r, s}^l(0) = \text{Saving}_{r, s}^l$$

$$\text{Balance}_{r, s}^l(t) = 
\text{Balance}_{r, s}^l(t-1) + 
\delta \text{Balance}_{r,s} ^ l(t)$$

$$\delta \text{Balance}_{r,s} ^ l(t) = 
\text{Earning}_{r, s}^c(t) - \text{Spending}_{r, s}^c(t)$$

Here, earnings are a function both of the employment status and initial earnings at the start of the simulation. We define a fixed parameter $\eta^c$ to introduce the impact on people's earning given different employment status. The corporate default rate provided by the bankruptcy model is used to simulate transitions between employment states.

$$\text{Earning}_{r, s}^l(t) =
\sum_c \lambda_{r, s}^c(t) * 
\eta^c * 
\text{Earning}_{r, s}^l(0)
$$

We further breakdown spendings into different expense sector($k$). Expenditure in each sector takes function of reduction in income and fear factor into account, bounded by the minimum level of spending to cover basic living cost. The expense from the population that are no longer capable of spending are carefully removed.

$$\text{Spending}_{r, s}^l(t) = \sum_k \text{Spending}_{r, s, k}^l(t)$$

$$\text{Spending}_{r, s, k}^l(t) =
\max(
\text{Spending}_{r, s, k}^l(0) * \frac{\text{Earning}_{r, s}^l(t)}{\text{Earning}_{r, s}^l(0)} * (1 - \text{Fear factor}),
\text{Minimum Spending}_{r, s, k}^l
) * (1 -  \lambda_{r, s}^\text{Dead}(t))
$$

We compare the spot spending at any point in time during the simulation to estimate the reduction in demand.

$$\text{Demand reduction}_k(t) = 1 - 
\frac
{\sum_r \sum_s \sum_l \text{Spending}_{r, s, k}^l(t)}
{\sum_r \sum_s \sum_l \text{Spending}_{r, s, k}^l(0)}$$

Note that this is not an agent-based model, i.e., we are not keeping track of balances by individual but only by group, which limits the ability of the model to simulate the effects of changes in employment status. A more granular modelling of these dynamics is left for future work.

Key limitation of the model is that apart from furloughing, benefits and Covid-19-specific government support programs are not taken into account. Also, as mentioned above, the labour market itself is not modelled, i.e., workers do not have the ability to switch jobs, or find new employment if their previous firm went bankrupt. There is currently no modelling of self-employment, part-time work or short-time work. Finally, while the individual insolvency model does explicitly simulate spending, this is not currently fed back to the demand parameter of the input-output model. All of these are areas of future work.

### Fear factor

In general, we expect people to be less motivated to go out and spend when they are mentally scared due to the coronavirus, even they are pysically health and well employed. Hence, we define a fear factor which linearly regresses on lockdown status, the number of ill people and the new dead cases.

$$\text{Fear factor}(t) = w_1 \text{Lock down}(t) + 
w_2 N_{\text{ill}}(t) + 
w_3 \delta N_{\text{dead}}(t)
$$

### Long-term Growth Model

The input-output model simulates economic output as a ratio of GDP at the start of the simulation. This ratio is then discounted based on corporate bankruptcies over time. Taken together, this produces a simulation of the shortfall in economic output due to the epidemic, at a short time horizon. By construction, it does not take growth dynamics into account. As a simple baseline model for economic growth we model the long-term GDP trend by assuming a fixed annual growth rate that persist throughout the simulation, except during lockdown periods. The level of economic output implied by this long term growth model is then multiplied by the discounted GDP ratio supplied by the shortfall model. Note that the shortfall model will dominate the results in the short term.

Modelling long-term growth is a key area for improvement in future work. On the one hand, the long-term growth model itself can be improved considerably, by adopting a more thorough autoregressive approach than assuming fixed growth. On the other hand, as mentioned above, a proper modelling of demand as part of the input-output model will provide more realistic long-term dynamics.

## Parameters

As referred to in the model description above, there are a large number of parameters on which the simulation model depends. These parameters need to be calibrated carefully for the mode to produce realistic results. **Importantly, the code currently does not provide default values for these parameters. Setting parameters is left to the user.** In some places, we may provide *randomized parameters* as a convenience for users when running the code. However, the outputs generated based on randomized parameters should *not* be interpreted as economically meaningful, either in qualitative or quantitative terms.

In some areas the current code does use parameters calibrated on data from the Office for National Statistics. This data is used on the basis of the Open Government License v3.0.

## References

1. Eurostat, *Eurostat Manual of supply, use and input-output tables*, Office for Official Publications of the European Communities, Luxembourg, 2008.
2. R. Wild, "United Kingdom Input-Output Analytical Tables 2010", Office for National Statistics, 2014.
3. H. Fadinger, C. Ghiglino, and M. Teteryatnikova, "Income Diﬀerences and Input-Output Structure", p. 55, 2016.
4. D. Acemoglu, V. M. Carvalho, A. Ozdaglar, and A. Tahbaz-Salehi, "The Network Origins of Aggregate Fluctuations", *Econometrica*, vol. 80, no. 5, pp. 1977–2016, 2012.
5. T. Mahmood, "Survival of newly founded businesses: A log-logistic model approach", *Small Business Economics*, vol. 14, no. 3, pp. 223-237, 2000.
6. P. Fisk, "The graduation of income distributions", Econometrica, vol. 29, no. 2, pp. 171-185, 1961.

## Disclaimers

Contains public sector information licensed under the [Open Government Licence v3.0](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/).

The data presented herein is solely for illustrative purposes and may include among other things modelling, back testing, simulated results and scenario analyses. The information is based upon certain factors, assumptions and historical information that Goldman Sachs may in its discretion have considered appropriate, however, Goldman Sachs provides no assurance or guarantee that this model will operate or would have operated in the past in a manner consistent with these assumptions. In the event any of the assumptions used do not prove to be true, results are likely to vary materially from the examples shown herein. Additionally, the results may not reflect material economic and market factors, which could impact results.

# Backlog

# Input-Output Model w/ Cobb-Douglas Production

## National Accounting

See the [ONS Article](http://www.ons.gov.uk/ons/guide-method/method-quality/specific/economy/input-output/articles-and-analyses/united-kingdom-input-output-analytical-tables-2010.pdf) as primary reference. See the [Eurostat Manual](https://ec.europa.eu/eurostat/documents/3859598/5902113/KS-RA-07-013-EN.PDF/b0b3d71e-3930-4442-94be-70b36cea9b39) for details. Data prepared as defined in the ONS Article is published by the ONS [here](https://www.ons.gov.uk/economy/nationalaccounts/supplyandusetables/datasets/ukinputoutputanalyticaltablesdetailed).

Note, the ONS data gives input-output tables product by product, rather than industry by industry. It does provide product-industry mappings as well as guidance on how to convert the data provided into industry by industry format. In the summary below I will therefore state everything product by product.

Restating key accounting identities from the paper as equations. We use variables with a tilde represent value of goods at base prices, see next section for details.

* $\tilde{d}_{i,j}$ input from product $i$ to product $j$ at base prices.
* $\tilde{q}_i$ total demand for product $i$, i.e., total output of product $i$ at base prices
* $\tilde{y}_i$ final use for product $i$ at base prices
* $\tilde{x}_i$ primary inputs to product $i$ at base prices

$$\underbrace{\text{total demand}_i}_{\tilde{q}_i} = \underbrace{\text{intermediate demand}_i}_{\sum_j {\tilde{d}_{i,j}}} + \underbrace{\text{final use}_i}_{\tilde{y}_i}$$

$$\underbrace{\text{final use}_i}_{\tilde{y}_i} = \text{final consumption}_i + \text{capital formation}_i + \text{exports}$$

$$\underbrace{\text{total demand}_i}_{\tilde{q}_i} = \underbrace{\text{intermediate inputs}_i}_{\sum_j \tilde{d}_{j,i}} + \underbrace{\text{primary inputs}_i}_{\tilde{x}_i}$$

$$ \text{primary inputs}_i = \text{imports}_i + \text{taxes on products}_i + \text{value add}_i$$

$$ \text{value add}_i = \text{taxes on production}_i + \text{compensation}_i + \text{gross operating surplus}_i$$

$$ \text{gross operating surplus}_i = \text{consumption of fixed capital}_i + \text{net operating surplus}_i$$

Note that **all of the quantities named above are given in ONS data**. 

Note also that there is also direct final use of imports, which is not reflected in the above. We will limit ourselves to considering imports only as inputs to domestic production for now.

To add some further granularity, it may be useful to break down net operating surplus further. However this breakdown is not given in the ONS IO Tables, and we will not use it in the version of the model described below.

$$\text{net operating surplus}_i = \text{interest}_i + \text{dividends}_i + \text{retained earnings}_i$$

$$\text{profit}_i = \text{dividends}_i + \text{retained earnings}_i$$

The crucial fact that, by construction, the total output of a given product is equal to the sum of all intermediate and primary inputs into that product is used for some key accounting identities.

$$\sum_j \tilde{d}_{i,j} + \tilde{y}_i = \sum_j \tilde{d}_{j,i} + \tilde{x}_i \text{ for all }i$$

$$\sum_i \tilde{x}_i = \sum_i \tilde{y}_i$$

$$\text{GDP} \approx \text{GVA} = \sum_i\text{value add}_i = \sum_i\text{final use}_i - \sum_i\text{imports}_i -\sum_i \text{taxes on products}_i$$

## From Accounting to Modelling

We draw on [Fadinger et. al "Income Difference and Input-Output Structure" (2016)](https://homepage.univie.ac.at/mariya.teteryatnikova/Research/FGT_lOstructure_CEPR.pdf).

We combine the above accounting identities with a Cobb-Douglas production function in order to model substitutability of inputs into production. While linear production functions allow perfect substitutions of inputs and $\min$ production functions do not allow any substitutions, Cobb-Douglas ensures that inputs can be substituted but at a cost.

Crucial difference between production functions and accounting identities is that production functions take as input the *quantity* of a given good, while accounting identities are stated in terms of market *value*, where $\text{value}=\text{price}\cdot\text{quantity}$.

We will make use of the values stated in the National Accounting section above as follows in the production function and the accounting function. 

| National Accounting | Production Function (qty) | Accounting Function (£) |
| --- | --- | --- |
| Imports | Imports (I) | Imports (I) |
| Taxes on products | - | Other |
| Taxes on production | - | Other |
| Compensation | Labour (L) | Compensation (L) |
| Consumption of fixed capital| Capital (K) | Cost of Capital (K) |
| Net Operating Surplus | Capital (K) | Cost of Capital (K)|


To set up the notation, we introduce a set of labels $M=\{I,L,K\}$, a set of labels $N$ for sectors with $n=|N|$ and variables

* $\tilde{x}_{m,i}$ value of primary input $m\in M$ to sector $i\in N$
* $\tilde{o}_i$ value of other primary inputs to sector $i$

On the final use side of things, we introduce labels $U=\{C,K,E\}$ to denote consumption, capital formation and exports, respectively and define $\tilde{y}_{u,i}$ to denote value of final use of type $u\in U$ of product $i$.

For variables $\tilde{d}_{i,j}$, $\tilde{x}_{m,i}$, $\tilde{y}_{u,i}$ and $\tilde{q}_i$ defined above, we introduce variables $d_{i,j}$, $x_{m,i}$, $y_{u,i}$ and $q_i$ that represent the associated *quantities*. To represent prices we define
* $p_i$ the price of one unit of good $i$, i.e., one unit of output sector $i$ produces,
* $p_m$ for $m\in M$ the price of one unit of primary input $m$.

With this notation for values prices and quantities, we have the identities
$$ \tilde{x}_{m,i} = p_m x_{m,i} $$
$$ \tilde{y}_{u,i} = p_i y_{u,i} $$
$$ \tilde{q}_{i} = p_i q_{i} $$
$$ \tilde{d}_{i,j} = p_i d_{i,j} $$

For example, $\tilde{x}_{L,i}$ denotes compensation while $x_{L,i}$ denotes quantity of labour.

The accounting identities above now become

$$ p_i q_i = \sum_j p_j d_{j,i} + \sum_m p_m x_{m,i} + o_i $$
$$ q_i = \sum_j d_{i,j} + \sum_u y_{u,i} $$

Note that the second identity doesn't need price coefficients as all quantities in the equation refer to the same good and therefore prices $p_i$ cancel. This is not the case for the first constraint as prices $p_j$ for different products $j$ appear.

To introduce the production function we define constants
* $\Lambda_i$ total factor productivity in sector $i$
* $\gamma_{i,j}$ and $\gamma_{m,j}$ for sectors $i,j\in N$ and primary inputs $m\in M$ such that $\sum_{j\in N} \gamma_{j,i} + \sum_{m\in M} \gamma_{m,i} = 1$.

The production function is given by the right-hand side of the constraint

$$ q_i = \Lambda_i \prod_j d_{j,i}^{\gamma_{j,i}} \prod_{m\in M} x^{\gamma_{m,i}}_{m,i}$$

Note that if we abbreviate $x_{L,i} = \ell_i$ and $x_{K,i}=k_i$ the form of the Cobb-Douglas production function may look more familiar. 

For interpretation, we can relate all of these variables back to the quantities defined in the previous section, as per the table above.

$$\tilde{x}_{I,i}=\text{imports}_i$$
$$\tilde{x}_{L,i}=\text{compensation}_i$$
$$o = \text{taxes on products}+ \text{taxes on production}$$
$$\tilde{x}_{K,i}= \text{consumption of fixed capital}_i + \text{net operating surplus}_i$$

Note that the production function constraint and the first accounting identity can hold at the same time, through adjustment of $\Lambda_i$, $p_i$ and $o_i$. We will avail ourselves of this flexibility when operationalizing the model below. In the Fadinger et al. paper, compare in particular Section 3.1, Definition, Item 2.

Labour is the key quantity we aim to constrain with the model. For this purpose, we introduce the following notation:

- $\lambda^{\text{ill}}_{i,r,a}$ ratio of workers ill/unable to work in sector $i$, region $r$, age $a$
- $\lambda^{\text{wfh}}_{i,r,a}$ ratio of workers working from home in sector $i$, region $r$, age $a$
- $\lambda^{\text{working}}_{i,r,a}$ ratio of workers at work in sector $i$, region $r$, age $a$
- $\lambda^{\text{furloughed}}_{i,r,a}$ ratio of workers furloughed in sector $i$, region $r$, age $a$
- $\lambda^{\text{unemployed}}_{i,r,a}$ ratio of workers unemployed in sector $i$, region $r$, age $a$

such that, for any triple $i,r,a$,

$$ \lambda^{\text{ill}}_{i,r,a} + \lambda^{\text{wfh}}_{i,r,a} + \lambda^{\text{working}}_{i,r,a} + \lambda^{\text{furloughed}}_{i,r,a} + \lambda^{\text{unemployment}}_{i,r,a} = 1$$

The input-output model above only works with sectors, not regions and age groups, so we aggregate 

$$\lambda^c_i = \sum_a \sum_r w_{i,r,a} \lambda^c_{i,r,a}$$

where $c\in\{\text{ill},\text{wfh},\text{working},\text{furloughed},\text{unemployed}\}$ and 
$w_{i,r,a}$ denotes the ratio of the number of workers in that sector, region and age group divided by the number of workers in that overall.

With all of the above in place, GDP, or more precisely GVA, can be obtained as follows:

$$ \text{GVA}_i = \tilde{x}_{L,i}+\tilde{x}_{K,i}+o_i-\text{taxes on products} = \tilde{y}_{i}-\tilde{x}_{I,i}-\text{taxes on products}$$

## Operationalizing the Model

The purpose of this section is to describe how we turn the relations described in the previous section into a model we can optimize over. In particular, we describe which symbols denote constants, parameters and variables and how we set constants based on available data.

### Data

For ease of notation, whenever we refer to a constant taken from a data source that refers to a quantity, we will make that clear with a superscript. For example, $d_{i,j}^{\text{iot}}$ would refer to the value of $d_{i,j}$ as given in the ONS's IO Tables, and $d_{i,j} \leq d_{i,j}^{\text{iot}}$ would express that $d_{i,j}$ in the model should be at most the constant $d_{i,j}^{\text{iot}}$.

From the ONS IO Tables, we can obtain $\tilde{d}_{i,j}^{\text{iot}}$, $\tilde{y}_{i}^{\text{iot}}$ and $\tilde{x}_{L,i}^{\text{iot}}$, $\tilde{x}_{I,i}^{\text{iot}}$ directly. We also define $$o_i^{\text{iot}}=\text{taxes on products}_i^{\text{iot}}+\text{taxes on production}_i^{\text{iot}}$$ and $$\tilde{x}_{K,i}^{\text{iot}} = \text{consumption of fixed capital}_i^{\text{iot}} + \text{net operating surplus}_i^{\text{iot}}.$$

### Prices, Labour and Capital

We set prices for **intermediate goods and imports**  to 1, i.e., equivalently, one unit of a good is by definition how much 1m GBP can buy. This in particular means we assume that prices of intermediate goods and imports are constant.

$$p_i = 1$$
$$p_I = 1$$

For **labour**, $p_L$ we be implied, per sector (!), by the constraints imposed by the utilization factors $\lambda$. Nevertheless, we assume that for the pre-crisis economy, the price of labour is 1 so that $\tilde{x}_{L,i}^{\text{iot}}=:x_{L,i}^{\text{iot}}$. This implicitly makes the modelling assumption that employees are not laid off and do not change jobs.

$$ \text{quantity of labour}_i = x_{L,i} = \left( 1\cdot \lambda^{\text{working}}_i + h_i\cdot\lambda^{\text{wfh}}_i + 0\cdot \lambda^{\text{sick}}_i + 0\cdot \lambda^{\text{furloughed}}_i + 0\cdot \lambda^{\text{unemployed}}_i \right) \tilde{x}_{L,i}^{\text{iot}} $$

$$ \text{compensation}_i = \tilde{x}_{L,i} = \left(1\cdot \lambda^{\text{working}}_i + 1\cdot\lambda^{\text{wfh}}_i + 1\cdot \lambda^{\text{sick}}_i + 0\cdot \lambda^{\text{furloughed}}_i + 0\cdot \lambda^{\text{unemployed}}_i\right) \tilde{x}_{L,i}^{\text{iot}}$$

In terms of model output, we are also interested in

$$ p_{L,i} = \frac{\tilde{x}_{L,i}}{x_{L,i}} $$
$$ \text{wages received}_i = \left(1\cdot \lambda^{\text{working}}_i + 1\cdot\lambda^{\text{wfh}}_i + 1\cdot \lambda^{\text{sick}}_i + 0.8\cdot \lambda^{\text{furloughed}}_i + 0\cdot \lambda^{\text{unemployed}}_i\right) \tilde{x}_{L,i}^{\text{iot}} $$

The latter assumes that the government subsidises furloughed workers, who are no longer either a cost to or a value add to the firm. Note, these identities are optional and need not be part of the model.

For **capital**, by contrast, we assume that capital is constant, while operating surplus is not. Therefore, return on captial $p_{K,i}$ is not constant either, but calibrated such that $p_{K,i}=1$ for the pre-crisis economy. Note: we could choose any other value here. Additionally, we introduce a parameter $\kappa_{i}$ that allows to scale the quantity of capital in a given sector, e.g., to model destruction of capital. This parameter is assumed to be given, i.e., exogenous to this model.

$$x_{K,i} = \kappa_i \tilde{x}^{\text{iot}}_{K,i}$$

The rationale for holding captial constant is that *in the short-term* no substantial investments in fixed capital will be possible to substitute for decrease in supply of (productive) labour. Substitution with working capital (goods/services) is still possible. In a secenario where depressed labour supply with limited ability to substitue leads to decreased output and therefore decreased operational surplus, this will translate, given the above constraints into decreasing return on capital.

It would be useful to produce profit as an additional output of the model, however that would require to break out interest paid from $x_{K,i}$ and no data on interest paid is given in the ONS IO Tables. This is left for future work.

### IO-Table weights

The weights $\gamma_{i,j}$ are defined as 
$$\gamma_{i,j} = \frac{\tilde{d}_{i,j}^{\text{iot}}}{\sum_i \tilde{d}_{i,j}^{\text{iot}} + \sum_m \tilde{x}_{m,j}^{\text{iot}}}$$
Note that we have omitted the term $o_i^{\text{iot}}$. Therefore, by construction, all the $\gamma_{i,j}$ appearing in the production function will sum to 1 as required.

### Taxes

We model taxes as proportional to output per sector, with the ratio determined by a parameter $\tau$ that is assumed as given. $\tau=1$ means the government does not cut tax rates, but taxes on both products and production scale proportionally to output. $\eta=0.3$ means taxes rates are cut to 30% of their current level. $\tau=0$ means there are no taxes at all.
    $$ o_i = o_i^\text{iot} \cdot \tau \cdot \frac{q_i}{q_i^{\text{iot}}} $$

Note that this implies that 
$$ \text{taxes on production}_i = \frac{\text{taxes on production}_i^{\text{iot}}}{o_i^{\text{iot}}}o_i 
= \tau q_i \frac{\text{taxes on production}_i^{\text{iot}}}{q_i^{\text{iot}}} $$

### Total Factor Productivity

Total factor productivity $\Lambda_i$ is defined such that the production function identity and the first accounting identity hold when all values are taken from the ONS IO Table.

$$\Lambda_i = \frac{
\frac{1}
              {1-
                  \frac{o_i^{\text{iot}}}
                       {q_i^{\text{iot}}}\tau
              }
\left(\sum_j \tilde{d}_{j,i}^{\text{iot}} + \sum_m \tilde{x}_{m,i}^{\text{iot}}\right)}{\prod_j (\tilde{d}_{j,i}^{\text{iot}})^{\gamma_{j,i}} \prod_{m\in M} (x^{\text{iot}}_{m,i})^{\gamma_{m,i}}}  $$

### Demand

We constrain demand per sector and type of final use relative to the pre-crisis level. These constraints are expected to be given as parameters $\delta_{u,i}$ for a given sector $i$ and final use type $u$.

The model does not currently account for which type of final use will receive available goods. Therefore, we pre-aggregate the bound on demand across difference types of final use and impose a single bound on total final use.

$$ y_{i}  \leq \sum_u \delta_{u,i} \tilde{y}_{u,i}^{\text{iot}} $$

### Simplification of Constraints

When applying the above constraints on prices to the key constrains simplify considerably.

$$ q_i = \Lambda_i \prod_j d_{j,i}^{\gamma_{j,i}} \prod_{m\in M} x^{\gamma_{m,i}}_{m,i}$$

$$ q_i = \frac{1}
              {1-
                  \frac{o_i^{\text{iot}}}
                       {q_i^{\text{iot}}}\tau
              }
              \left(\sum_j d_{j,i} + x_{I,i} + \tilde{x}_{L,i} + \tilde{x}_{K,i}\right) $$
$$ q_i = \sum_j d_{i,j} + y_i $$

A crucial question to consider is how the first two equations above can hold simultaneously, given that prices have been fixed and total factor productivity is taken as constant. Furthermore the relationship between $\tilde{x}_{L,i}$ and $x_{L,i}$ is fixed and $o_i$ is taken to be constant. The only degree of freedom left is that in the production equation we use $x_{K,i}$ whereas in the accounting identity we use $\tilde{x}_{K,i}$ and we do not impose any constraint on $p_K$. This means that any difference between the right-hand sides of the first two equations will be reflected in $\frac{\tilde{x}_{L,i}}{x_{L,i}}$ which is the return on capital. Again, compare Section 3.1, Definition, Item 2 in Fadinger et al.

## Model

Summarizing all of the above in concise form.

### Parameters / Inputs

#### Labour

- $\lambda^{\text{ill}}_{i,r,a}$ ratio of workers ill/unable to work in sector $i$, region $r$, age $a$
- $\lambda^{\text{wfh}}_{i,r,a}$ ratio of workers working from home in sector $i$, region $r$, age $a$
- $\lambda^{\text{working}}_{i,r,a}$ ratio of workers at work in sector $i$, region $r$, age $a$
- $\lambda^{\text{furloughed}}_{i,r,a}$ ratio of workers furloughed in sector $i$, region $r$, age $a$
- $\lambda^{\text{unemployed}}_{i,r,a}$ ratio of workers unemployed in sector $i$, region $r$, age $a$

such that, for any triple $i,r,a$,

$$ \lambda^{\text{ill}}_{i,r,a} + \lambda^{\text{wfh}}_{i,r,a} + \lambda^{\text{working}}_{i,r,a} + \lambda^{\text{furloughed}}_{i,r,a} + \lambda^{\text{unemployment}}_{i,r,a} = 1$$

These inputs are preprocessed as follows, for $c\in\{\text{working},\text{wfh},\text{ill},\text{furloughed},\text{unemployed}\}$:

$$\lambda^c_i = \sum_a \sum_r w_{i,r,a} \lambda^c_{i,r,a}$$

#### Capital

* $\kappa_i$ ratio of fixed capital, relative to the pre-crisis level, available in sector $i$

#### Demand

* $\delta_{C,i}$ ratio of consumption in sector $i$, relative to pre-crisis level
* $\delta_{K,i}$ ratio of capital formation in sector $i$, relative to pre-crisis level
* $\delta_{E,i}$ ratio of exports in sector $i$, relative to pre-crisis level

abbreviated as $\delta_{u,i}$ for $u\in\{C, K, E\}$.

Note these are used as *bounds* on demand, i.e., the demand output by the model may be lower. The demand estimated by the model is given only at the aggregate level per sector, i.e., $y_i =\sum_u y_{u,i}$.

#### Taxes

* $\tau$ tax *rate* on both products and production, expressed as a ratio of the pre-crisis level

### Variables

Using the notation from above, variables in the below are $q_i$, $d_{i,j}$, $x_{I,i}$, $x_{L,i}$, $x_{K,i}$, $\tilde{x}_{L,i}$, $\tilde{x}_{K,i}$, $y_{i}$.

### Constants

$$\gamma_{i,j} = \frac{\tilde{d}_{i,j}^{\text{iot}}}{\sum_i \tilde{d}_{i,j}^{\text{iot}} + \sum_m \tilde{x}_{m,j}^{\text{iot}}}$$

$$\gamma_{m,j} = \frac{\tilde{x}_{m,j}^{\text{iot}}}{\sum_i \tilde{d}_{i,j}^{\text{iot}} + \sum_m \tilde{x}_{m,j}^{\text{iot}}}$$

$$\Lambda_i = \frac{
\frac{1}
              {1-
                  \frac{o_i^{\text{iot}}}
                       {q_i^{\text{iot}}}\tau
              }
\left(\sum_j \tilde{d}_{j,i}^{\text{iot}} + \sum_m \tilde{x}_{m,i}^{\text{iot}}\right)}{\prod_j (\tilde{d}_{j,i}^{\text{iot}})^{\gamma_{j,i}} \prod_{m\in M} (x^{\text{iot}}_{m,i})^{\gamma_{m,i}}}  $$

### Objective

$$ \max \sum_i \tilde{x}_{L,i} + \tilde{x}_{K,i} + \tau q_i \frac{\text{taxes on production}_i^{\text{iot}}}{q_i^{\text{iot}}} $$

### Constraints

$$ q_i = \Lambda_i \prod_j d_{j,i}^{\gamma_{j,i}} \prod_{m\in M} x^{\gamma_{m,i}}_{m,i}$$

$$ q_i = \frac{1}
              {1-
                  \frac{o_i^{\text{iot}}}
                       {q_i^{\text{iot}}}\tau
              }
              \left(\sum_j d_{j,i} + x_{I,i} + \tilde{x}_{L,i} + \tilde{x}_{K,i}\right) $$
$$ q_i = \sum_j d_{i,j} + \sum_u y_{u,i} $$

$$ x_{L,i} = \left( 1\cdot \lambda^{\text{working}}_i + h_i\cdot\lambda^{\text{wfh}}_i + 0\cdot \lambda^{\text{ill}}_i + 0\cdot \lambda^{\text{furloughed}}_i + 0\cdot \lambda^{\text{unemployed}}_i \right) \tilde{x}_{L,i}^{\text{iot}} $$

$$ \tilde{x}_{L,i} = \left(1\cdot \lambda^{\text{working}}_i + 1\cdot\lambda^{\text{wfh}}_i + 1\cdot \lambda^{\text{ill}}_i + 0\cdot \lambda^{\text{furloughed}}_i + 0\cdot \lambda^{\text{unemployed}}_i\right) \tilde{x}_{L,i}^{\text{iot}}$$

$$\lambda^{\text{working}}_i + \lambda^{\text{wfh}}_i +  \lambda^{\text{ill}}_i +  \lambda^{\text{furloughed}}_i +  \lambda^{\text{unemployed}}_i = 1$$

$$\lambda^{\text{furloughed}}_i = p_{\text{furloughed}} (\lambda^{\text{furloughed}}_i + \lambda^{\text{unemployed}}_i)$$

$$ \lambda^{\text{ill employed}} = p_{\text{ill employed}}(\lambda^{\text{working}}_i + \lambda^{\text{wfh}}_i + \lambda^{\text{ill}}) $$

$$ \lambda^{\text{wfh}} = p_{\text{wfh}}(\lambda^{\text{working}}_i + \lambda^{\text{wfh}}_i) $$

$$p_{\text{unemployed}} = \lambda^{\text{furloughed}}_i + \lambda^{\text{unemployed}}_i $$

IO Model

Input:
* $p_{\text{ill employed}}$ proportion of employed people unable to work, driven by spread model, 
* $p_{\text{wfh}}$ proportion of employed people restricted to working from home, driven by government restrictions and corp policy
* $p_{\text{furloughed}}$ proportion of not employed people in the workforce who are furloughed, driven by government intervention
* $p_{\text{dead}}$ proportion of people in the workforce who have died

Output:
* $p_{\text{unemployed}}$ proportion of workforce unemployed

Slices:
* Need sector 
* Optional: also region and age


Health

* well
* ill
* dead

Economics

* Employed
    * Working at Work
    * Working from Home
* Not Employed
    * Furloughed
    * Unemployed

Categories

* Well X Economic Categories (4)
* Ill X {Employed, Furloughed, Unemployed} (3)
* Dead (1)


$$x_{K,i} = \kappa_i \tilde{x}^{\text{iot}}_{K,i}$$

$$ y_{i} \leq \sum_u \delta_{u,i} \tilde{y}_{u,i}^{\text{iot}} $$

### Bounds

* $0\leq q_i$
* $0\leq d_{i,j}$
* $0 \leq x_{I,i}$
* $0 \leq x_{L,i}$
* $0 \leq x_{K,i}$
* $0 \leq \tilde{x}_{L,i}$
* $\tilde{x}_{K,i}$ is unbounded
* $0 \leq y_i$

### Outputs

GDP, more precisely GVA, in total and by sector, expressed as a ratio of pre-crisis GDP.

$$ \text{GVA}_i = \frac{\tilde{x}_{L,i}+\tilde{x}_{K,i}+\tau q_i \frac{\text{taxes on production}_i^{\text{iot}}}{q_i^{\text{iot}}}}{\tilde{x}_{L,i}^\text{iot}+\tilde{x}_{K,i}^\text{iot}+\text{taxes on production}^\text{iot}} $$

$$ \text{GVA} = \sum_i \text{GVA}_i$$